In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
from torch import multiprocessing

In [ ]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

In [ ]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

channel = EngineConfigurationChannel()
env_path = "D:/_Thesis/warehouse-bot-training/environment_builds/warehouse_stage2_find/Warehouse_Bot.exe"

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  # additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=1)

#### Transform environment from `mlagents` to `gymnasium`

In [ ]:
import gymnasium as gym
print(gym.__version__)

In [ ]:
from env_camera_raycasts_gymnasium_wrapper import UnityCameraRaycastsGymWrapper 
from env_raycasts_gymnasium_Wrapper import UnityRaycastsGymWrapper 

# gymnasium_env = UnityRaycastsGymWrapper(unity_env)
gymnasium_env = UnityCameraRaycastsGymWrapper(unity_env)

#### Creating stable_baselines3 model

In [ ]:
from stable_baselines3 import PPO

model = PPO.load("./saved_models/baselines/stage2/find_1_1M", gymnasium_env)

In [ ]:
print(model.policy)

In [ ]:
env = model.get_env()
obs = env.reset()

In [ ]:
import time

for i in range(5000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    # time.sleep(0.5)